# Data Cleaning

In this temporary file, I want to clean the data. All the procedures below are written into dataframe_utils.py as a function clean_dataframe(df) and impute_data(df).

- First there is missing data. This is a small percentage and can be dealt with accordingly. But I will first delete data that is improbable, and then fix the missing data. This would ensure that while filling missing data with averages, we do not consider the outliers.
- z-coordinate cannot be negative.
- It is not possible that the server and the receiver are on the same side of the court. These points must be deleted.
- It is not possible that the server and the ball bounce is on the same side of the court. These points must be deleted.
- <b>Fliping the court</b> to make things simpler. I.e the server will always be on the left and the returner will always be on the right. This shouldn't make any difference to the game.
- Server position must be close to the baseline and cannot be way inside the court or way behind.
- Returner position must be near or behind the baseline and cannot be way inside the court.
- Fastest serve speed recorded is 163.7mph. Serve speed cannot exceed this.
- ball_net_z of 1.5m or above will be deleted. It's not probable that serves that are in go too high above the net.
- Delete outliers based on Inter Quartile Range of training data.
- Impute missing data with mean and most frequent.

## Importing libraries

In [7]:
# Import everything from my_libraries.py
from my_libraries import *

In [8]:
df = pd.read_csv("datasets/task_set.csv")
df

,surface,serve_side,serve_number,ball_hit_y,ball_hit_x,ball_hit_z,ball_hit_v,ball_net_v,ball_net_z,ball_net_y,ball_bounce_x,ball_bounce_y,ball_bounce_v,ball_bounce_angle,hitter_x,hitter_y,receiver_x,receiver_y,hitter_hand,receiver_hand,is_ace
0,hard,deuce,2,1.459,11.445,2.830,91.5,67.9,1.241,-0.980,-4.392,-2.278,60.7,19.216,11.739,1.398,-13.343,-3.570,right,left,0
1,hard,ad,1,1.083,-11.228,2.817,125.5,97.7,1.019,-0.026,4.528,-0.618,88.8,12.428,-11.744,1.131,15.459,-4.067,left,right,0
2,hard,deuce,1,0.835,11.149,2.793,109.7,85.5,1.156,-2.082,-5.378,-3.744,75.3,12.737,11.538,0.647,-13.679,-3.544,right,right,0
3,clay,ad,2,-1.424,11.488,2.727,97.1,74.1,1.095,1.170,-3.915,1.853,68.1,17.743,11.690,-1.169,-11.378,3.402,right,right,0
4,hard,ad,1,0.472,-10.985,2.660,121.0,95.7,1.094,-0.301,5.267,-0.525,85.5,12.013,-11.508,0.559,14.323,-3.629,right,right,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187921,hard,deuce,2,0.303,11.230,2.725,92.6,73.2,1.230,-1.833,-4.921,-3.108,65.6,15.935,11.573,0.361,-15.278,-3.645,right,right,0
187922,hard,ad,1,0.886,-11.358,2.813,112.3,86.1,1.083,-2.094,4.636,-3.113,76.6,13.120,-11.615,0.844,13.219,-3.299,right,right,0
187923,hard,ad,1,1.395,-11.254,2.782,104.0,78.7,1.140,-2.276,4.499,-3.556,70.5,16.639,-11.625,1.050,14.470,-3.828,right,right,0
187924,clay,deuce,1,0.623,11.179,2.806,130.4,104.1,1.203,-0.735,-6.308,-1.568,92.2,10.420,11.616,0.988,-14.068,-3.368,right,right,0


In [9]:
df.describe()

,serve_number,ball_hit_y,ball_hit_x,ball_hit_z,ball_hit_v,ball_net_v,ball_net_z,ball_net_y,ball_bounce_x,ball_bounce_y,ball_bounce_v,ball_bounce_angle,hitter_x,hitter_y,receiver_x,receiver_y,is_ace
count,187926.000000,187817.000000,187817.000000,187817.000000,187807.000000,187797.000000,187809.000000,187810.000000,187817.000000,187817.000000,187809.000000,187816.000000,187810.000000,187810.000000,187717.000000,187717.00000,187926.000000
mean,1.356635,-0.003056,0.025959,2.842908,108.650497,85.511926,1.177421,-0.002502,-0.015045,-0.002121,75.887740,14.625971,0.073049,-0.034099,-0.044800,-0.01033,0.080585
std,0.479007,1.148724,11.362744,0.147901,15.495405,13.023081,0.162155,1.392535,5.146167,2.415050,12.496144,3.643096,11.671999,1.167514,14.066468,3.67635,0.272197
min,1.000000,-7.593000,-19.079000,-0.006000,2.800000,0.000000,-21.292000,-9.194000,-27.580000,-9.170000,0.000000,0.000000,-19.997000,-7.819000,-19.961000,-8.01000,0.000000
25%,1.000000,-1.028000,-11.371000,2.772000,96.700000,75.200000,1.084000,-1.103000,-5.086000,-2.062000,66.600000,12.191000,-11.669000,-1.088000,-14.077000,-3.67300,0.000000
50%,1.000000,-0.035000,9.600000,2.844000,109.700000,86.400000,1.163000,-0.002000,-3.180000,0.000000,76.400000,14.052000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,2.000000,1.024000,11.371000,2.918000,121.100000,96.000000,1.250000,1.112000,5.082000,2.078000,85.700000,16.969000,11.727000,1.028000,14.053000,3.65100,0.000000
max,2.000000,6.511000,17.802000,4.213000,195.300000,163.000000,11.637000,8.563000,23.332000,8.319000,164.600000,90.000000,19.709000,8.007000,19.989000,8.25900,1.000000


In [10]:
# Count missing values per column
missing_count_per_column = df.isna().sum()
print(missing_count_per_column)

surface                0
serve_side             0
serve_number           0
ball_hit_y           109
ball_hit_x           109
ball_hit_z           109
ball_hit_v           119
ball_net_v           129
ball_net_z           117
ball_net_y           116
ball_bounce_x        109
ball_bounce_y        109
ball_bounce_v        117
ball_bounce_angle    110
hitter_x             116
hitter_y             116
receiver_x           209
receiver_y           209
hitter_hand            0
receiver_hand          0
is_ace                 0
dtype: int64


## Cleaning

In [11]:
# Check for duplicates
duplicates = df.duplicated()
# Print the number of duplicate rows
print(f"Number of duplicate rows: {duplicates.sum()}")

Number of duplicate rows: 82


In [12]:
# Remove duplicate rows
df = df.drop_duplicates()

In [13]:
# z-coordinates cannot be negative at all.
df = df[(df["ball_hit_z"] >= 0) & (df["ball_net_z"] >= 0)]

In [14]:
# It is not possible that the server and the receiver are on the same side of the court. These points must be deleted.
df = df[
    ((df["ball_hit_x"] < 0) & (df["receiver_x"] > 0))
    | ((df["ball_hit_x"] > 0) & (df["receiver_x"] < 0))
]

In [15]:
# It is not possible that the server and the ball bounce is on the same side of the court. These points must be deleted.
df = df[
    ((df["ball_bounce_x"] < 0) & (df["hitter_x"] > 0))
    | ((df["ball_bounce_x"] > 0) & (df["hitter_x"] < 0))
]

In [16]:
# I will also flip the courts to make things simpler. Ie the server will always be on the left and the returner will always be on the
# right. This shouldn't make any difference to the game.

# If ball hit is on the right, ball_hit_x is positive.
# Change sign of ball_hit_x and ball_hit_y to retain ad/deuce court.
# Change sign of ball_net_y
# Change sign of ball_bounce_x, ball_bounce_y
# Change sign of hitter_x, hitter_y
# Change sign of receiver_x, receiver_y

# Multiply the column to be flipped by -1 if 'ball_hit_x' is positive.
cols_to_flip = [
    "ball_hit_x",
    "ball_hit_y",
    "ball_net_y",
    "ball_bounce_x",
    "ball_bounce_y",
    "hitter_x",
    "hitter_y",
    "receiver_x",
    "receiver_y",
]
df.loc[df["ball_hit_x"] > 0, cols_to_flip] *= -1

In [17]:
# Ball placement has to be inside the service box and not outside.
# Adding 10cm tolerance due to thickness of the line and ball slippage.
df = df[(df["ball_bounce_x"] <= service_line_x + 0.1)]
df = df[
    (df["ball_bounce_y"] >= -singles_sideline_y - 0.1)
    & (df["ball_bounce_y"] <= singles_sideline_y + 0.1)
]

In [18]:
# Server position must be close to the baseline and cannot be way inside the court or way behind.
# Allowing 1 m behind baseline (generous) and 0.5m within baesline, and 1m within sidelines.
df = df[(df["hitter_x"] >= -baseline_x - 1)]
df = df[
    (df["hitter_y"] >= -singles_sideline_y + 1)
    & (df["hitter_y"] <= singles_sideline_y - 1)
]

In [19]:
df

,surface,serve_side,serve_number,ball_hit_y,ball_hit_x,ball_hit_z,ball_hit_v,ball_net_v,ball_net_z,ball_net_y,ball_bounce_x,ball_bounce_y,ball_bounce_v,ball_bounce_angle,hitter_x,hitter_y,receiver_x,receiver_y,hitter_hand,receiver_hand,is_ace
0,hard,deuce,2,-1.459,-11.445,2.830,91.5,67.9,1.241,0.980,4.392,2.278,60.7,19.216,-11.739,-1.398,13.343,3.570,right,left,0
1,hard,ad,1,1.083,-11.228,2.817,125.5,97.7,1.019,-0.026,4.528,-0.618,88.8,12.428,-11.744,1.131,15.459,-4.067,left,right,0
2,hard,deuce,1,-0.835,-11.149,2.793,109.7,85.5,1.156,2.082,5.378,3.744,75.3,12.737,-11.538,-0.647,13.679,3.544,right,right,0
3,clay,ad,2,1.424,-11.488,2.727,97.1,74.1,1.095,-1.170,3.915,-1.853,68.1,17.743,-11.690,1.169,11.378,-3.402,right,right,0
4,hard,ad,1,0.472,-10.985,2.660,121.0,95.7,1.094,-0.301,5.267,-0.525,85.5,12.013,-11.508,0.559,14.323,-3.629,right,right,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187921,hard,deuce,2,-0.303,-11.230,2.725,92.6,73.2,1.230,1.833,4.921,3.108,65.6,15.935,-11.573,-0.361,15.278,3.645,right,right,0
187922,hard,ad,1,0.886,-11.358,2.813,112.3,86.1,1.083,-2.094,4.636,-3.113,76.6,13.120,-11.615,0.844,13.219,-3.299,right,right,0
187923,hard,ad,1,1.395,-11.254,2.782,104.0,78.7,1.140,-2.276,4.499,-3.556,70.5,16.639,-11.625,1.050,14.470,-3.828,right,right,0
187924,clay,deuce,1,-0.623,-11.179,2.806,130.4,104.1,1.203,0.735,6.308,1.568,92.2,10.420,-11.616,-0.988,14.068,3.368,right,right,0


In [20]:
# Returner position must be near or behind the baseline and cannot be way inside the court.
# Allowing 7 m behind baseline (generous) and 2m within baesline, and 1.5m within sidelines.
df = df[(df["receiver_x"] <= baseline_x + 7) & (df["receiver_x"] >= baseline_x - 2)]
df = df[
    (df["receiver_y"] >= -singles_sideline_y - 1.5)
    & (df["receiver_y"] <= singles_sideline_y + 1.5)
]

In [21]:
# Fastest serve speed recorded is 163.7mph. Serve speed cannot exceed this.
df = df[df["ball_hit_v"] <= 163.7]

In [22]:
df.shape

(182356, 21)

In [23]:
# ball_net_z of 1.5m or above will be deleted. It's not probable that serves that are in go too high above the net.
# And height should be greater than 0.915m
df = df[(df["ball_net_z"] < 1.5) & (df["ball_net_z"] >= 0.915)]

In [24]:
# It is not possible for ball speed to be greater at the net than when the server has just hit the ball. These points must be deleted.
df = df[df["ball_hit_v"] > df["ball_net_v"]]

In [25]:
df.columns

Index(['surface', 'serve_side', 'serve_number', 'ball_hit_y', 'ball_hit_x',
       'ball_hit_z', 'ball_hit_v', 'ball_net_v', 'ball_net_z', 'ball_net_y',
       'ball_bounce_x', 'ball_bounce_y', 'ball_bounce_v', 'ball_bounce_angle',
       'hitter_x', 'hitter_y', 'receiver_x', 'receiver_y', 'hitter_hand',
       'receiver_hand', 'is_ace'],
      dtype='object')

In [ ]:
df

In [26]:
# Deleting points outside IQR bounds
columns = [
    "ball_hit_y",
    "ball_hit_x",
    "ball_hit_z",
    "ball_hit_v",
    "ball_net_v",
    "ball_net_z",
    "ball_net_y",
    "ball_bounce_x",
    "ball_bounce_y",
    "ball_bounce_v",
    "ball_bounce_angle",
    "hitter_x",
    "hitter_y",
    "receiver_x",
    "receiver_y",
]
train_df = pd.read_csv("datasets/train_dataset.csv")
# Compute IQR using TRAINING data
Q1_train = train_df[columns].quantile(0.25)
Q3_train = train_df[columns].quantile(0.75)
IQR_train = Q3_train - Q1_train

# Define bounds using training data
lower_bound = Q1_train - 1.5 * IQR_train
upper_bound = Q3_train + 1.5 * IQR_train

for col in columns:
    df = df[(df[col] >= lower_bound[col]) & (df[col] <= upper_bound[col])]

In [ ]:
df

In [27]:
# Count missing values per column
missing_count_per_column = df.isna().sum()
print(missing_count_per_column)

surface              0
serve_side           0
serve_number         0
ball_hit_y           0
ball_hit_x           0
ball_hit_z           0
ball_hit_v           0
ball_net_v           0
ball_net_z           0
ball_net_y           0
ball_bounce_x        0
ball_bounce_y        0
ball_bounce_v        0
ball_bounce_angle    0
hitter_x             0
hitter_y             0
receiver_x           0
receiver_y           0
hitter_hand          0
receiver_hand        0
is_ace               0
dtype: int64


In [19]:
# Create an imputer that fills missing values with the most frequent category
imputer = SimpleImputer(strategy="most_frequent")

cols_most_frequent = [
    "surface",
    "serve_side",
    "serve_number",
    "hitter_hand",
    "receiver_hand",
]
df[cols_most_frequent] = imputer.fit_transform(df[cols_most_frequent])

# Save the fitted imputer
joblib.dump(imputer, "simple_imputer.joblib")

['simple_imputer.joblib']

In [20]:
# Impute missing values with column mean (only for numeric columns)
df.fillna(df.mean(numeric_only=True), inplace=True)